# Advanced DiD Analysis: Modern Estimators and Heterogeneity

**Objective**: Apply modern difference-in-differences methods robust to treatment effect heterogeneity in staggered adoption settings.

**Key additions**:
- Goodman-Bacon decomposition to understand TWFE bias
- Callaway-Sant'Anna estimator for robust ATT estimation
- Comprehensive heterogeneity analysis
- Market regime analysis (bull vs bear markets)

In [1]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path

from src.data_loader import IPODataLoader
from src.estimators import TWFEEstimator, EventStudyEstimator, test_parallel_trends
from src.modern_did import GoodmanBacon, CallawayEstimator

import warnings
warnings.filterwarnings('ignore')

# Setup output directories
fig_output_dir = Path("../outputs/figures")
fig_output_dir.mkdir(parents=True, exist_ok=True)

results_output_dir = Path("../outputs/results")
results_output_dir.mkdir(parents=True, exist_ok=True)

## 1. Data Loading

In [2]:
loader = IPODataLoader()
df = loader.load_stock_data(market_adjusted=True)
df_clean = loader.prepare_panel_data(df)

print(f"Data: {len(df_clean):,} observations, {df_clean['Ticker'].nunique()} companies")
print(f"Treatment: {df_clean['Post_Lockup'].sum():,} post-lockup observations")

Data: 17,802 observations, 71 companies
Treatment: 9,022 post-lockup observations


## 2. Baseline TWFE Estimation

Standard two-way fixed effects estimator for comparison.

In [3]:
twfe = TWFEEstimator()
result_twfe = twfe.estimate(df_clean)

print(f"TWFE Estimate: {result_twfe.coefficient:.4f}%")
print(f"Standard Error: {result_twfe.std_error:.4f}%")
print(f"P-value: {result_twfe.p_value:.4f}")
print(f"95% CI: [{result_twfe.ci_lower:.4f}%, {result_twfe.ci_upper:.4f}%]")

TWFE Estimate: 0.4545%
Standard Error: 0.1289%
P-value: 0.0004
95% CI: [0.2019%, 0.7072%]


## 3. Goodman-Bacon Decomposition

Decompose the TWFE estimate to understand potential bias from heterogeneous treatment effects.

**Key insight**: In staggered DiD, TWFE uses already-treated units as controls for late-treated units. This can bias estimates if treatment effects evolve over time.

In [4]:
# Add treatment time variable
df_clean['Treatment_Time'] = df_clean.groupby('Ticker')['Days_Since_IPO'].transform(
    lambda x: (x > 180).idxmax() if (x > 180).any() else np.inf
)

gb = GoodmanBacon(
    df_clean,
    outcome='Abnormal_Return',
    group_var='Ticker',
    time_var='Days_Since_IPO',
    treatment_time_var='Treatment_Time'
)

decomp = gb.decompose()

print("\nGoodman-Bacon Decomposition:")
print(decomp.groupby('comparison_type')[['weight', 'att']].agg({
    'weight': 'sum',
    'att': 'mean'
}).round(4))

# Visualize
fig = go.Figure()
for comp_type in decomp['comparison_type'].unique():
    subset = decomp[decomp['comparison_type'] == comp_type]
    fig.add_trace(go.Scatter(
        x=subset['att'],
        y=subset['weight'],
        mode='markers',
        name=comp_type,
        marker=dict(size=10)
    ))

fig.update_layout(
    title='Goodman-Bacon Decomposition: Weight vs ATT',
    xaxis_title='ATT Estimate (%)',
    yaxis_title='Weight in TWFE',
    template='plotly_white'
)
fig.show()
fig.write_image(f"{fig_output_dir}/06_goodman_bacon_decomposition.png", scale=2)


Goodman-Bacon Decomposition:
                                        weight     att
comparison_type                                       
Earlier vs Later Treated                   0.5 -0.2618
Later vs Earlier Treated (Problematic)     0.5     NaN


## 4. Callaway-Sant'Anna Estimation (Not Applicable)

**Note**: The Callaway-Sant'Anna estimator requires a never-treated control group or not-yet-treated units. In our IPO setting, all companies experience lockup expiration (universal treatment), so this modern DiD method is not applicable. 

**Alternative approaches for this setting**:
- Event study design (implemented in notebooks 02-03)
- Stacked DiD with cohort-specific controls
- Synthetic control methods
- Instrumental variables using variation in lockup length

We proceed with the event study and heterogeneity analysis instead.

In [5]:
# Callaway-Sant'Anna not applicable - all units treated
# Skipping this analysis

print("Callaway-Sant'Anna estimator requires never-treated units.")
print("In this IPO setting, all companies experience lockup expiration.")
print("\nContinuing with event study and heterogeneity analysis...")

Callaway-Sant'Anna estimator requires never-treated units.
In this IPO setting, all companies experience lockup expiration.

Continuing with event study and heterogeneity analysis...


## 5. Event Study with Confidence Intervals

In [6]:
es = EventStudyEstimator()
event_results = es.estimate(df_clean, pre_window=60, post_window=60)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=event_results['event_time'],
    y=event_results['coefficient'],
    mode='lines+markers',
    name='Point Estimate',
    line=dict(color='#2E86AB', width=2),
    marker=dict(size=5)
))

fig.add_trace(go.Scatter(
    x=event_results['event_time'],
    y=event_results['ci_upper'],
    mode='lines',
    line=dict(width=0),
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=event_results['event_time'],
    y=event_results['ci_lower'],
    mode='lines',
    fill='tonexty',
    line=dict(width=0),
    name='95% CI',
    fillcolor='rgba(46, 134, 171, 0.2)'
))

fig.add_hline(y=0, line_dash="dash", line_color="gray")
fig.add_vline(x=0, line_dash="solid", line_color="red", annotation_text="Lockup Expires")

fig.update_layout(
    title='Event Study: Dynamic Treatment Effects',
    xaxis_title='Days Relative to Lockup Expiration',
    yaxis_title='Abnormal Return (%)',
    template='plotly_white',
    height=500
)
fig.show()
fig.write_image(f"{fig_output_dir}/07_event_study_with_ci.png", scale=2)

## 6. Heterogeneity Analysis

### 6.1 By Company Size

In [7]:
characteristics = loader.get_company_characteristics(df_clean)
df_with_char = df_clean.merge(characteristics[['Ticker', 'Size_Category']], on='Ticker')

results_by_size = {}
for size in ['Small', 'Medium', 'Large']:
    df_subset = df_with_char[df_with_char['Size_Category'] == size]
    if len(df_subset) > 100:
        result = twfe.estimate(df_subset)
        results_by_size[size] = result

print("\nTreatment Effects by Company Size:")
for size, result in results_by_size.items():
    sig = "***" if result.p_value < 0.01 else "**" if result.p_value < 0.05 else "*" if result.p_value < 0.10 else ""
    print(f"{size:8s}: {result.coefficient:+.4f}% (SE: {result.std_error:.4f}) {sig}")


Treatment Effects by Company Size:
Small   : -0.1419% (SE: 0.2139) 
Medium  : +0.4252% (SE: 0.2001) **
Large   : +0.7555% (SE: 0.2382) ***


### 6.2 By IPO Year (Market Regime)

In [8]:
ipo_meta = loader.load_ipo_metadata()
df_with_year = df_clean.merge(ipo_meta[['Ticker', 'IPO_Year']], on='Ticker')

# Group years into market regimes
df_with_year['Market_Regime'] = df_with_year['IPO_Year'].map({
    2018: 'Bull (2018-2019)',
    2019: 'Bull (2018-2019)',
    2020: 'COVID (2020-2021)',
    2021: 'COVID (2020-2021)',
    2022: 'Bear (2022-2024)',
    2023: 'Bear (2022-2024)',
    2024: 'Bear (2022-2024)'
})

results_by_regime = {}
for regime in df_with_year['Market_Regime'].dropna().unique():
    df_subset = df_with_year[df_with_year['Market_Regime'] == regime]
    if len(df_subset) > 100:
        result = twfe.estimate(df_subset)
        results_by_regime[regime] = result

print("\nTreatment Effects by Market Regime:")
for regime, result in results_by_regime.items():
    sig = "***" if result.p_value < 0.01 else "**" if result.p_value < 0.05 else "*" if result.p_value < 0.10 else ""
    print(f"{regime:20s}: {result.coefficient:+.4f}% (SE: {result.std_error:.4f}) {sig}")


Treatment Effects by Market Regime:
COVID (2020-2021)   : +0.6145% (SE: 0.1512) ***
Bear (2022-2024)    : +0.9120% (SE: 0.6345) 
Bull (2018-2019)    : +0.1543% (SE: 0.1717) 


## 7. Placebo Analysis Interpretation

The placebo tests from Notebook 03 showed significant effects at fake lockup dates. Let's investigate why.

In [9]:
# Test multiple placebo dates
placebo_days = list(range(60, 301, 30))
placebo_results = []

for day in placebo_days:
    df_temp = df_clean.copy()
    df_temp['Post_Lockup'] = (df_temp['Days_Since_IPO'] > day).astype(int)
    
    result = twfe.estimate(df_temp)
    placebo_results.append({
        'day': day,
        'coefficient': result.coefficient,
        'p_value': result.p_value,
        'is_real': day == 180
    })

df_placebo = pd.DataFrame(placebo_results)

# Visualize
fig = go.Figure()

real = df_placebo[df_placebo['is_real']]
fake = df_placebo[~df_placebo['is_real']]

fig.add_trace(go.Scatter(
    x=fake['day'],
    y=fake['coefficient'],
    mode='markers',
    name='Placebo',
    marker=dict(size=8, color='lightgray')
))

fig.add_trace(go.Scatter(
    x=real['day'],
    y=real['coefficient'],
    mode='markers',
    name='Real Lockup (Day 180)',
    marker=dict(size=12, color='red', symbol='star')
))

fig.add_hline(y=0, line_dash="dash", line_color="gray")

fig.update_layout(
    title='Placebo Tests: Effect Estimates at Real vs Fake Lockup Dates',
    xaxis_title='Days Since IPO',
    yaxis_title='Estimated Effect (%)',
    template='plotly_white'
)
fig.show()
fig.write_image(f"{fig_output_dir}/08_placebo_multiple_dates.png", scale=2)

print("\nInterpretation:")
sig_placebos = (fake['p_value'] < 0.05).sum()
print(f"Significant placebo effects: {sig_placebos} out of {len(fake)}")
print(f"\nThis suggests the effect may reflect:")
print(f"1. General IPO lifecycle dynamics (prices evolve over first year)")
print(f"2. Gradual information incorporation rather than discrete shock")
print(f"3. Survivorship effects (weak IPOs exit before lockup)")
print(f"\nCausal interpretation should be cautious.")


Interpretation:
Significant placebo effects: 5 out of 8

This suggests the effect may reflect:
1. General IPO lifecycle dynamics (prices evolve over first year)
2. Gradual information incorporation rather than discrete shock
3. Survivorship effects (weak IPOs exit before lockup)

Causal interpretation should be cautious.


## 8. Summary and Recommendations

### Key Findings

1. **TWFE Estimate**: +0.45% (p<0.001)
2. **Goodman-Bacon Decomposition**: Reveals composition of TWFE estimate
3. **Modern DiD Methods**: Not applicable due to universal treatment (all IPOs have lockup expiration)
4. **Heterogeneity**: Effect varies by company size and market regime
5. **Placebo Tests**: Multiple fake dates show effects, suggesting broader IPO lifecycle dynamics

### Limitations

1. **Identification**: All units treated → no never-treated control group
2. **Insider selling volume**: Unobserved
3. **Economic significance**: Effect small relative to IPO volatility
4. **Placebo results**: Suggest identification challenges
5. **Sample**: Limited to tech IPOs 2018-2024